In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graph

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
!wget -q http://apache.mirrors.hoobly.com/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

In [ ]:
!ls

drive	     spark-2.4.7-bin-hadoop2.7	    spark-2.4.7-bin-hadoop2.7.tgz.1
sample_data  spark-2.4.7-bin-hadoop2.7.tgz


In [ ]:
!tar -xvf spark-2.4.7-bin-hadoop2.7.tgz

spark-2.4.7-bin-hadoop2.7/
spark-2.4.7-bin-hadoop2.7/kubernetes/
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/py_container_checks.py
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/pyfiles.py
spark-2.4.7-bin-hadoop2.7/kubernetes/tests/worker_memory_check.py
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/entrypoint.sh
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/python/Dockerfile
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/bindings/R/Dockerfile
spark-2.4.7-bin-hadoop2.7/kubernetes/dockerfiles/spark/Dockerfile
spark-2.4.7-bin-hadoop2.7/NOTICE
spark-2.4.7-bin-hadoop2.7/data/
spark-2.4.7-bin-hadoop2.7/data/strea

In [ ]:
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.7-bin-hadoop2.7"

## Starting Point: SparkSession

SparkSession in Spark 2.0 provides builtin support for Hive features including the ability to write queries using HiveQL, access to Hive UDFs, and the ability to read data from Hive tables. To use these features, you do not need to have an existing Hive setup.

In [ ]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [ ]:
from pyspark import SparkContext, SparkConf
sc = SparkContext.getOrCreate()

In [ ]:
from io import StringIO

In [ ]:
from pyspark.sql.types import StructType
import datetime

In [ ]:
df = spark.read.format('com.databricks.spark.csv').\
                        options(header='true', \
                        inferschema='true').\
              load("/content/drive/MyDrive/IDS 561/project/publicsafety-chicago-crime/chicago_crime_" + str(2014) + ".csv",header=True);
#df.select('date').toPandas().tail(100)
print((df.count(), len(df.columns)))

(274423, 17)


In [ ]:
year = [2015,2016]
for i in year:
  print(i)
  dfyear = spark.read.format('com.databricks.spark.csv').\
                        options(header='true', \
                        inferschema='true').\
              load("/content/drive/MyDrive/IDS 561/project/publicsafety-chicago-crime/chicago_crime_" + str(i) + ".csv",header=True);
  df = df.union(dfyear)
print((df.count(), len(df.columns)))

2015
2016
(754541, 17)


In [ ]:
from pyspark.sql.functions import *
from pyspark.sql import functions as f

In [ ]:
df = df.withColumn("date", f.from_unixtime(f.unix_timestamp("date",'MM/dd/yyyy hh:mm:ss aa'),'MM/dd/yyyy HH:mm:ss'))\
.withColumn('date', f.to_timestamp('date', 'MM/dd/yyyy HH:mm:ss'))\
.withColumn('dateonly', f.to_date(f.col("date")))\
.withColumn('year', f.date_format('date', 'yyyy'))\
.withColumn('month', f.date_format('date', 'MM'))\
.withColumn('day', f.date_format('date', 'dd'))\
.withColumn('dayofweek', f.dayofweek('date'))\
.withColumn('timeonly', f.date_format('date', 'HH:mm:ss'))\
.withColumn('hour', f.date_format('date', 'HH'))


In [ ]:
df = df.withColumn('daypart',\
              f.when((f.col("hour") >= "06") & (f.col("hour") < "09"), "early morning")\
              .when((f.col("hour") >= "09") & (f.col("hour") < "12"), "late morning")\
              .when((f.col("hour") >= "12") & (f.col("hour") < "15"), "early afternoon")\
              .when((f.col("hour") >= "15") & (f.col("hour") < "18"), "late afternoon")\
              .when((f.col("hour") >= "18") & (f.col("hour") < "21"), "evening")\
              .when((f.col("hour") >= "21") & (f.col("hour") < "24"), "night")\
              .when((f.col("hour") >= "00") & (f.col("hour") < "03"), "late night")\
              .otherwise("toward morning"))

In [ ]:
df = df.withColumn('season',\
              f.when((f.col("month").isin(['03','04','05'])), "spring")\
              .when((f.col("month").isin(['06','07','08'])), "summer")\
              .when((f.col("month").isin(['09','10','11'])), "fall")\
              .otherwise("winter"))

In [ ]:
df = df.withColumnRenamed("Primary Type","PrimaryType").\
withColumnRenamed("Location Description","LocationDescription").\
withColumnRenamed("Community Area","CommunityArea").\
withColumnRenamed("FBI Code","FBICode")

# Predict arrest

In [ ]:
df.show(1)

+-------+-----------+-------------------+-------------------+----+-----------+-----------+-------------------+------+--------+----+--------+----+-------------+-------+------------+-------------+----------+----+-----+---+---------+--------+----+----------+------+
|     ID|Case Number|               date|              Block|IUCR|PrimaryType|Description|LocationDescription|Arrest|Domestic|Beat|District|Ward|CommunityArea|FBICode|    Latitude|    Longitude|  dateonly|year|month|day|dayofweek|timeonly|hour|   daypart|season|
+-------+-----------+-------------------+-------------------+----+-----------+-----------+-------------------+------+--------+----+--------+----+-------------+-------+------------+-------------+----------+----+-----+---+---------+--------+----+----------+------+
|9446824|   HX100141|2014-01-01 02:00:00|0000X W ILLINOIS ST|0460|    BATTERY|     SIMPLE|             STREET| false|   false|1831|      18|  42|            8|    08B|41.890828047|-87.630234746|2014-01-01|2014| 

In [ ]:
df.schema.names

['ID',
 'Case Number',
 'date',
 'Block',
 'IUCR',
 'PrimaryType',
 'Description',
 'LocationDescription',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'CommunityArea',
 'FBICode',
 'Latitude',
 'Longitude',
 'dateonly',
 'year',
 'month',
 'day',
 'dayofweek',
 'timeonly',
 'hour',
 'daypart',
 'season']

variables needed to be removed: ID, Case Number, date, Block, IUCR,CommunityArea, Latitude, Longitude,dateonly, year, timeonly

In [ ]:
deselect_features = [
 'Description',                    
 'ID',
 'Case Number',
 'date',
 'Block',
 'IUCR',
 'Latitude',
 'Longitude',
 'dateonly',
 'year',
 'timeonly',
 'area_name']

In [ ]:
features_df = df.drop(*deselect_features)
features_df.printSchema()

root
 |-- PrimaryType: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: boolean (nullable = true)
 |-- Domestic: boolean (nullable = true)
 |-- Beat: integer (nullable = true)
 |-- District: integer (nullable = true)
 |-- Ward: integer (nullable = true)
 |-- CommunityArea: integer (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dayofweek: integer (nullable = true)
 |-- hour: string (nullable = true)
 |-- daypart: string (nullable = false)
 |-- season: string (nullable = false)



In [ ]:
from pyspark.sql.types import StringType
from pyspark.sql.functions import col

In [ ]:
# features_df = features_df.withColumn("Arrest", col("Arrest").cast('string')) \
# .withColumn("Domestic", col("Domestic").cast('string'))\
# .withColumn("Beat", col("Beat").cast('float'))\
# .withColumn("District", col("District").cast('float'))\
# .withColumn("Ward", col("Ward").cast('float'))\
# .withColumn("CommunityArea", col("CommunityArea").cast('float'))\
# .withColumn("month", col("month").cast('float'))\
# .withColumn("day", col("day").cast('float'))\
# .withColumn("dayofweek", col("dayofweek").cast('float'))\
# .withColumn("hour", col("hour").cast('float'))\
# .withColumn("dayofweek", col("dayofweek").cast('float'))\

In [ ]:
features_df = features_df.withColumn("Arrest", col("Arrest").cast('string')) \
.withColumn("Domestic", col("Domestic").cast('string'))\
.withColumn("Beat", col("Beat").cast('float'))\
.withColumn("District", col("District").cast('string'))\
.withColumn("Ward", col("Ward").cast('string'))\
.withColumn("CommunityArea", col("CommunityArea").cast('string'))\
.withColumn("month", col("month").cast('string'))\
.withColumn("day", col("day").cast('string'))\
.withColumn("dayofweek", col("dayofweek").cast('string'))\
.withColumn("hour", col("hour").cast('string'))\
.withColumn("dayofweek", col("dayofweek").cast('string'))\

In [ ]:
features_df.printSchema()

root
 |-- PrimaryType: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: float (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- CommunityArea: string (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dayofweek: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- daypart: string (nullable = false)
 |-- season: string (nullable = false)



In [ ]:
features_df.show()

+-----------------+--------------------+------+--------+------+--------+----+-------------+-------+-----+---+---------+----+----------+------+
|      PrimaryType| LocationDescription|Arrest|Domestic|  Beat|District|Ward|CommunityArea|FBICode|month|day|dayofweek|hour|   daypart|season|
+-----------------+--------------------+------+--------+------+--------+----+-------------+-------+-----+---+---------+----+----------+------+
|          BATTERY|              STREET| false|   false|1831.0|      18|  42|            8|    08B|   01| 01|        4|  02|late night|winter|
|            THEFT|       BAR OR TAVERN| false|   false|1832.0|      18|  42|            8|     06|   01| 01|        4|  00|late night|winter|
|  CRIMINAL DAMAGE|           APARTMENT| false|   false|1523.0|      15|  28|           25|     14|   01| 01|        4|  00|late night|winter|
|          BATTERY|           APARTMENT|  true|   false|2514.0|      25|  31|           19|    08B|   01| 01|        4|  00|late night|winter|

In [ ]:
#PrimaryType,Description,LocationDescription,Arrest,Domestic,Beat

In [ ]:
features_df.columns

['PrimaryType',
 'LocationDescription',
 'Arrest',
 'Domestic',
 'Beat',
 'District',
 'Ward',
 'CommunityArea',
 'FBICode',
 'month',
 'day',
 'dayofweek',
 'hour',
 'daypart',
 'season']

In [ ]:
# feature_level_count_dict = []
# for feature in features_df.columns:
#     levels_list_df = features_df.select(feature).distinct()
#     feature_level_count_dict.append({'feature': feature, 'level_count': levels_list_df.count()})

In [ ]:
# import pandas as pd

# pd.DataFrame(feature_level_count_dict).sort_values(by='level_count', ascending=False)

In [ ]:
# indices = [0,1,2,3,4,9,-2,-1]
# selected_elements = []
# for index in indices:
#     selected_elements.append(feature_level_count_dict[index])

In [ ]:
# selected_elements

In [ ]:
# features_df.show(1)

In [ ]:
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler, VectorIndexer

In [ ]:
features_df = features_df.replace('?', None).dropna(how='any')

In [ ]:
#print((df.count(), len(df.columns)))

In [ ]:
#print((features_df.count(), len(features_df.columns)))

In [ ]:
# features_df.show(1)

In [ ]:
# required_index = ['PrimaryType',
#                   'LocationDescription',
#                   'Arrest',
#                   'Domestic',
#                   'FBICode',
#                   'daypart',
#                   'season']

In [ ]:
# for feature in required_index:
#   print(feature)

PrimaryType
LocationDescription
Arrest
Domestic
FBICode
daypart
season


In [ ]:
for feature in features_df.columns:
    indexer = StringIndexer()\
                 .setInputCol (feature)\
                 .setOutputCol (feature+"_indexed")
    print('Fitting feature "%s"' % feature)
    model = indexer.fit(features_df)
    print('Transforming "%s"' % feature)
    features_df = model.transform(features_df)

Fitting feature "PrimaryType"
Transforming "PrimaryType"
Fitting feature "LocationDescription"
Transforming "LocationDescription"
Fitting feature "Arrest"
Transforming "Arrest"
Fitting feature "Domestic"
Transforming "Domestic"
Fitting feature "Beat"
Transforming "Beat"
Fitting feature "District"
Transforming "District"
Fitting feature "Ward"
Transforming "Ward"
Fitting feature "CommunityArea"
Transforming "CommunityArea"
Fitting feature "FBICode"
Transforming "FBICode"
Fitting feature "month"
Transforming "month"
Fitting feature "day"
Transforming "day"
Fitting feature "dayofweek"
Transforming "dayofweek"
Fitting feature "hour"
Transforming "hour"
Fitting feature "daypart"
Transforming "daypart"
Fitting feature "season"
Transforming "season"


In [ ]:
features_df.show()

+-----------------+--------------------+------+--------+------+--------+----+-------------+-------+-----+---+---------+----+----------+------+-------------------+---------------------------+--------------+----------------+------------+----------------+------------+---------------------+---------------+-------------+-----------+-----------------+------------+---------------+--------------+
|      PrimaryType| LocationDescription|Arrest|Domestic|  Beat|District|Ward|CommunityArea|FBICode|month|day|dayofweek|hour|   daypart|season|PrimaryType_indexed|LocationDescription_indexed|Arrest_indexed|Domestic_indexed|Beat_indexed|District_indexed|Ward_indexed|CommunityArea_indexed|FBICode_indexed|month_indexed|day_indexed|dayofweek_indexed|hour_indexed|daypart_indexed|season_indexed|
+-----------------+--------------------+------+--------+------+--------+----+-------------+-------+-----+---+---------+----+----------+------+-------------------+---------------------------+--------------+-----------

In [ ]:
features_df.printSchema()

In [ ]:
# features_df = features_df.drop('PrimaryType_indexed_onehoted')

In [ ]:
# required_onehot = ['PrimaryType_indexed','LocationDescription_indexed','Domestic_indexed','Beat','District','Ward','CommunityArea','FBICode_indexed','month','day','dayofweek','hour','daypart_indexed','season_indexed']

In [ ]:
required_onehot = ['PrimaryType_indexed','LocationDescription_indexed','Domestic_indexed','Beat_indexed','District_indexed','Ward_indexed','CommunityArea_indexed','FBICode_indexed','month_indexed','day_indexed','dayofweek_indexed','hour_indexed','daypart_indexed','season_indexed']

In [ ]:
for feature in required_onehot:
    OneHot = OneHotEncoderEstimator()\
    .setInputCols([feature])\
    .setOutputCols([feature+'_onehoted'])
    print('Fitting feature "%s"' % feature)
    model = OneHot.fit(features_df)
    print('Transforming "%s"' % feature)
    features_df = model.transform(features_df)

Fitting feature "PrimaryType_indexed"
Transforming "PrimaryType_indexed"
Fitting feature "LocationDescription_indexed"
Transforming "LocationDescription_indexed"
Fitting feature "Domestic_indexed"
Transforming "Domestic_indexed"
Fitting feature "Beat_indexed"
Transforming "Beat_indexed"
Fitting feature "District_indexed"
Transforming "District_indexed"
Fitting feature "Ward_indexed"
Transforming "Ward_indexed"
Fitting feature "CommunityArea_indexed"
Transforming "CommunityArea_indexed"
Fitting feature "FBICode_indexed"
Transforming "FBICode_indexed"
Fitting feature "month_indexed"
Transforming "month_indexed"
Fitting feature "day_indexed"
Transforming "day_indexed"
Fitting feature "dayofweek_indexed"
Transforming "dayofweek_indexed"
Fitting feature "hour_indexed"
Transforming "hour_indexed"
Fitting feature "daypart_indexed"
Transforming "daypart_indexed"
Fitting feature "season_indexed"
Transforming "season_indexed"


In [ ]:
features_df.printSchema()

root
 |-- PrimaryType: string (nullable = true)
 |-- LocationDescription: string (nullable = true)
 |-- Arrest: string (nullable = true)
 |-- Domestic: string (nullable = true)
 |-- Beat: float (nullable = true)
 |-- District: string (nullable = true)
 |-- Ward: string (nullable = true)
 |-- CommunityArea: string (nullable = true)
 |-- FBICode: string (nullable = true)
 |-- month: string (nullable = true)
 |-- day: string (nullable = true)
 |-- dayofweek: string (nullable = true)
 |-- hour: string (nullable = true)
 |-- daypart: string (nullable = true)
 |-- season: string (nullable = true)
 |-- PrimaryType_indexed: double (nullable = false)
 |-- LocationDescription_indexed: double (nullable = false)
 |-- Arrest_indexed: double (nullable = false)
 |-- Domestic_indexed: double (nullable = false)
 |-- Beat_indexed: double (nullable = false)
 |-- District_indexed: double (nullable = false)
 |-- Ward_indexed: double (nullable = false)
 |-- CommunityArea_indexed: double (nullable = false)
 

In [ ]:
required_features = ['PrimaryType_indexed_onehoted',
                    'LocationDescription_indexed_onehoted',
                    'Domestic_indexed_onehoted',
                    'Beat_indexed_onehoted',
                    'District_indexed_onehoted',
                    'Ward_indexed_onehoted',
                    'CommunityArea_indexed_onehoted',
                    'FBICode_indexed_onehoted',
                    'month_indexed_onehoted',
                    'day_indexed_onehoted',
                    'dayofweek_indexed_onehoted',
                    'hour_indexed_onehoted',
                    'daypart_indexed_onehoted',
                    'season_indexed_onehoted',
                   ]

In [ ]:
# required_features = ['PrimaryType_indexed',
#                     'LocationDescription_indexed',
#                     'Domestic_indexed',
#                     'Beat',
#                     'District',
#                     'Ward',
#                     'CommunityArea',
#                     'FBICode_indexed',
#                     'month',
#                     'day',
#                     'dayofweek',
#                     'hour',
#                     'daypart_indexed',
#                     'season_indexed',
#                    ]

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=required_features, outputCol='NumFeatures')

transformed_data = assembler.transform(features_df)

standardscaler = StandardScaler(inputCol="NumFeatures", outputCol="features")

In [ ]:
transformed_data.show()

+-----------------+--------------------+------+--------+------+--------+----+-------------+-------+-----+---+---------+----+----------+------+-------------------+---------------------------+--------------+----------------+------------+----------------+------------+---------------------+---------------+-------------+-----------+-----------------+------------+---------------+--------------+----------------------------+------------------------------------+-------------------------+---------------------+-------------------------+---------------------+------------------------------+------------------------+----------------------+--------------------+--------------------------+---------------------+------------------------+-----------------------+--------------------+
|      PrimaryType| LocationDescription|Arrest|Domestic|  Beat|District|Ward|CommunityArea|FBICode|month|day|dayofweek|hour|   daypart|season|PrimaryType_indexed|LocationDescription_indexed|Arrest_indexed|Domestic_indexed|Beat_

In [ ]:
scaler_model=standardscaler.fit(transformed_data)

transformed_data = scaler_model.transform(transformed_data)

# Modeling

In [ ]:
# Split the data
(training_data, test_data) = transformed_data.randomSplit([0.6,0.4], seed=42)

# NAIVE BAYES

In [ ]:
from pyspark.ml.classification import NaiveBayes

# create the trainer and set its parameters
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol='Arrest_indexed', featuresCol='features')

# train the model
model = nb.fit(training_data)

# select example rows to display.
predictions = model.transform(test_data)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Make predicitons
predictionAndTarget = model.transform(test_data).select("Arrest_indexed", "prediction")

predictionAndTargetNumpy = np.array((predictionAndTarget.collect()))

acc = accuracy_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('accuracy:',acc)
f1 = f1_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('f1:',f1)
precision = precision_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('weightedPrecision:',precision)
recall = recall_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('weightedRecall:',recall)
auc = roc_auc_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('auc:',auc)

accuracy: 0.7649834043783204
f1: 0.5967344704916537
weightedPrecision: 0.5236689646891666
weightedRecall: 0.6934951700743371
auc: 0.7412007502227874


# RF

In [ ]:
# Define the model
from pyspark.ml.classification import RandomForestClassifier

rf = RandomForestClassifier(labelCol='Arrest_indexed', 
                            featuresCol='features')

In [ ]:
# Fit the model
model = rf.fit(training_data)

In [ ]:
# Predict with the test dataset
predictions = model.transform(test_data)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Make predicitons
predictionAndTarget = model.transform(test_data).select("Arrest_indexed", "prediction")

predictionAndTargetNumpy = np.array((predictionAndTarget.collect()))

acc = accuracy_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
f1 = f1_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
precision = precision_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
recall = recall_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
auc = roc_auc_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])

In [ ]:
print('accuracy:',acc)
print('f1:',f1)
print('weightedPrecision:',precision)
print('weightedRecall:',recall)
print('auc:',auc)

accuracy: 0.8359475452101945
f1: 0.5173266600846538
weightedPrecision: 0.9861732259988074
weightedRecall: 0.350630076722276
auc: 0.6744924803164759


# Logistic

In [ ]:
from pyspark.ml.classification import LogisticRegression
logisticRegression = LogisticRegression(labelCol='Arrest_indexed', 
                                        featuresCol='features', 
                                        maxIter=10, 
                                        family='multinomial',
                                        regParam=0.0)
fittedModel = logisticRegression.fit(training_data)

In [ ]:
# Predict with the test dataset
logisticRegressionpredictions = fittedModel.transform(test_data)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Make predicitons
predictionAndTarget = fittedModel.transform(test_data).select("Arrest_indexed", "prediction")

predictionAndTargetNumpy = np.array((predictionAndTarget.collect()))

acc = accuracy_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
f1 = f1_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
precision = precision_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
recall = recall_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
auc = roc_auc_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])

In [ ]:
print('accuracy:',acc)
print('f1:',f1)
print('weightedPrecision:',precision)
print('weightedRecall:',recall)
print('auc:',auc)

accuracy: 0.8660422681679696
f1: 0.673019374406968
weightedPrecision: 0.8673494983277592
weightedRecall: 0.5498297269004996
auc: 0.7608449083251145


# SVM

In [ ]:
from pyspark.ml.classification import LinearSVC

lsvc = LinearSVC(labelCol='Arrest_indexed',
                 featuresCol='features',
                 maxIter=10, 
                 regParam=0.1)

# train the model
model = lsvc.fit(training_data)

# select example rows to display.
predictions = model.transform(test_data)

In [ ]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, f1_score

# Make predicitons
predictionAndTarget = model.transform(test_data).select("Arrest_indexed", "prediction")

predictionAndTargetNumpy = np.array((predictionAndTarget.collect()))

acc = accuracy_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('accuracy:',acc)
f1 = f1_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('f1:',f1)
precision = precision_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('weightedPrecision:',precision)
recall = recall_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('weightedRecall:',recall)
auc = roc_auc_score(predictionAndTargetNumpy[:,0], predictionAndTargetNumpy[:,1])
print('auc:',auc)

accuracy: 0.856948731323696
f1: 0.611954288186308
weightedPrecision: 0.9566344501113021
weightedRecall: 0.44986550412763193
auc: 0.7215205773179
